In [1]:
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
from keras.preprocessing.image import ImageDataGenerator
import time

Using TensorFlow backend.


In [2]:
rescale = 1./255
IMG_HEIGHT = 150
IMG_WIDTH = 150
MINI_BATCH = 32
PATH = "/Users/hyunjaecho/Desktop/chest_xray/"

train_generator = ImageDataGenerator(rescale=rescale).flow_from_directory(
    PATH + "train",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=MINI_BATCH,
    class_mode='binary')

v_generator = ImageDataGenerator(rescale=rescale).flow_from_directory(
    PATH + "val",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=MINI_BATCH,
    class_mode='binary')

test_generator = ImageDataGenerator(rescale=rescale).flow_from_directory(
    PATH + "test",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=MINI_BATCH,
    class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
# from PIL import Image

# img = Image.fromarray(train_generator[0][0][:,:,0], 'RGB')
# img.save('my.png')
# img.show()

In [4]:
# Variables

NUM_CHANNELS = 3
NUM_LABELS = 2
LEARNING_RATE = 0.01

with tf.name_scope("hidden-layer"):
    conv1_weights = tf.Variable(tf.truncated_normal([7, 7, NUM_CHANNELS, 32]))
    conv1_bias = tf.Variable(tf.zeros([32]))

    conv2_weights = tf.Variable(tf.truncated_normal([7, 7, 32, 64]))
    conv2_bias = tf.Variable(tf.zeros([64]))

    fc1_weights = tf.Variable(tf.truncated_normal([int(np.ceil(IMG_HEIGHT / 4)) * int(np.ceil(IMG_WIDTH / 4)) * 64, 512]))
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[512]))

    fc2_weights = tf.Variable(tf.truncated_normal([512, NUM_LABELS]))
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS]))


In [5]:
def model(data):
    # conv layer 1
    print("DATA: ", data.shape)
    conv = tf.nn.conv2d(data, filter=conv1_weights, strides=[1,1,1,1], padding='SAME')
    conv = tf.nn.bias_add(conv, conv1_bias)
    activate = tf.nn.relu(conv)
    dropout = tf.nn.dropout(activate, 0.5)
    pool = tf.nn.max_pool(dropout, ksize=[1,2,2,1], strides=[1, 2, 2, 1], padding='SAME')

    # conv layer 2
    conv = tf.nn.conv2d(pool, filter=conv2_weights, strides=[1,1,1,1], padding='SAME')
    conv = tf.nn.bias_add(conv, conv2_bias)
    activate = tf.nn.relu(conv)
    dropout = tf.nn.dropout(activate, 0.5)
    
    pool = tf.nn.max_pool(activate, ksize=[1,2,2,1], strides=[1, 2, 2, 1], padding='SAME')
    pool_shape = pool.get_shape().as_list()
    pool = tf.convert_to_tensor(pool)
    
    reshape = tf.reshape(pool, [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])

    print("POOL: ", pool_shape)
    print("reshape: ", reshape.shape)
    print("FC1 W: ",fc1_weights.get_shape().as_list())
    print(int(np.ceil(IMG_HEIGHT / 4)), int(np.ceil(IMG_WIDTH / 4)))
    
    # fc layer 1
    fc1 = tf.matmul(reshape, fc1_weights) + fc1_biases
    fc1_activate = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1_activate, 0.5)
    
    # fc layer 2
    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases
    
    return fc2

In [6]:
# placeholders
with tf.name_scope("placeholders"):
    train_batch = tf.placeholder(dtype=tf.float32, shape=[MINI_BATCH, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS])
    train_labels = tf.placeholder(dtype=tf.int64, shape=[MINI_BATCH,])
    
    eval_batch = tf.placeholder(dtype=np.float32, shape=[16, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS])
    eval_labels = tf.placeholder(dtype=np.int64, shape=[16,])
    
    

In [7]:
logits = model(train_batch)

# Predictions for the current training minibatch.
train_prediction = tf.nn.softmax(logits)

# Validation prediction
eval_prediction = tf.nn.softmax(model(eval_batch))

with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=train_labels, logits=logits))
    regularizers = (tf.nn.l2_loss(fc1_weights)
                + tf.nn.l2_loss(fc1_biases)
                + tf.nn.l2_loss(fc2_weights)
                + tf.nn.l2_loss(fc2_biases))
    
    alpha = 5e-2
    loss += alpha * regularizers
    
with tf.name_scope("optimizer"):
    optimizer = tf.train.MomentumOptimizer(0.01, 0.9).minimize(loss)


DATA:  (32, 150, 150, 3)
<class 'tensorflow.python.framework.ops.Tensor'>
POOL:  [32, 38, 38, 64]
reshape:  (32, 92416)
FC1 W:  [92416, 512]
38 38
DATA:  (16, 150, 150, 3)
<class 'tensorflow.python.framework.ops.Tensor'>
POOL:  [16, 38, 38, 64]
reshape:  (16, 92416)
FC1 W:  [92416, 512]
38 38


In [8]:
def error_rate(predictions, labels):
  """Return the error rate based on dense predictions and sparse labels."""
  return 100.0 - (
      100.0 *
      np.sum(np.argmax(predictions, 1) == labels) /
      predictions.shape[0])

In [12]:
# Training

"""
NORMAL = 0
PNEUMONIA = 1

train_generator: 163 of minibatches of training data
train_generator[i] = 32 images
train_generator[i][j] = i,j -th image, shape (IMG_HEIGHT, IMG_WIDTH, CHANNELS=3)

"""

NUM_ITERATIONS = 100
start_time = time.time()

with tf.Session() as sess:
    print("Session started")
    sess.run(tf.global_variables_initializer())
    
    with tf.name_scope("summaries"):
        tf.summary.scalar("loss", loss)
        writer = tf.summary.FileWriter("./nn_logs", sess.graph)
        merged = tf.summary.merge_all()
    
    for i in np.arange(NUM_ITERATIONS):
        
        batch_data = train_generator[i][0]
        batch_labels = train_generator[i][1]
        
        feed_dict = {train_batch:batch_data, train_labels:batch_labels}

        sess.run(optimizer, feed_dict=feed_dict)
        
        if i % 10 == 0:
            print(i, "-th iteration")
            l, predictions = sess.run([loss, train_prediction], feed_dict=feed_dict)
            print("training loss: ", l)
            print("training predictions: ", error_rate(predictions, batch_labels))
            
            # Validation
            val_data, val_labels = v_generator[0][0], v_generator[0][1]
            val_feed_dict = {eval_batch:val_data, eval_labels:val_labels}
            val_prediction = sess.run(eval_prediction, feed_dict=val_feed_dict)
#             print("\nvalidation loss: ", v_l)
            print("validation predictions: ", error_rate(val_prediction, val_labels))

    print("Training: ", time.time() - start_time)
    
    
#     test_error = error_rate(test_data, test_labels)
#     print('Test error: %.1f%%' % test_error)
        
        

Session started
0 -th iteration
training loss:  2568509.5
training predictions:  18.75
validation predictions:  50.0
10 -th iteration
training loss:  31848.5
training predictions:  31.25
validation predictions:  37.5
20 -th iteration
training loss:  8699.543
training predictions:  25.0
validation predictions:  25.0
30 -th iteration
training loss:  2515.264
training predictions:  21.875
validation predictions:  18.75
40 -th iteration
training loss:  5131.893
training predictions:  18.75
validation predictions:  31.25
50 -th iteration
training loss:  1828.672
training predictions:  15.625
validation predictions:  12.5
60 -th iteration
training loss:  87.83351
training predictions:  3.125
validation predictions:  25.0
70 -th iteration
training loss:  1209.5933
training predictions:  21.875
validation predictions:  31.25
80 -th iteration
training loss:  569.9312
training predictions:  12.5
validation predictions:  12.5
90 -th iteration
training loss:  374.8001
training predictions:  6.25
v